<a href="https://colab.research.google.com/github/parotnes/Course-AI/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import random
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:


# Input and output paths
input_folder = '/content/drive/MyDrive/drone_data'
output_folder = '/content/drive/MyDrive/drone_data/new_images'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to read YOLOv8 label file and extract bounding box coordinates
def read_yolo_label(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()

    # Assuming one object per image for simplicity
    label = lines[0].strip().split(' ')
    class_id = int(label[0])
    x_center, y_center, width, height = map(float, label[1:])

    return class_id, x_center, y_center, width, height

# Function to crop the image based on YOLOv8 label
def crop_image(image_path, label_path, output_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    class_id, x_center, y_center, bbox_width, bbox_height = read_yolo_label(label_path)

    # Calculate bounding box coordinates in pixel values
    x1 = int((x_center - bbox_width / 2) * width)
    y1 = int((y_center - bbox_height / 2) * height)
    x2 = int((x_center + bbox_width / 2) * width)
    y2 = int((y_center + bbox_height / 2) * height)

    # Crop the image based on the bounding box
    cropped_image = image[y1:y2, x1:x2]

    # Save the cropped image
    cv2.imwrite(output_path, cropped_image)

# Function to randomly crop a portion of the image
def random_crop_image(image_path, output_path, crop_size=(224, 224)):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Randomly select top-left coordinates for cropping
    top_left_x = random.randint(0, width - crop_size[1])
    top_left_y = random.randint(0, height - crop_size[0])

    # Crop the image
    cropped_image = image[top_left_y:top_left_y + crop_size[0], top_left_x:top_left_x + crop_size[1]]

    # Save the cropped image
    cv2.imwrite(output_path, cropped_image)

# Process each image and corresponding label
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        image_path = os.path.join(input_folder, filename)
        label_path = os.path.join(input_folder, os.path.splitext(filename)[0] + ".txt")

        if os.path.exists(label_path):
            # Process image based on YOLOv8 label
            output_path_yolo = os.path.join(output_folder, "yolo_" + filename)
            crop_image(image_path, label_path, output_path_yolo)

            # Process image with random crop
            output_path_random = os.path.join(output_folder, "random_" + filename)
            random_crop_image(image_path, output_path_random)




In [ ]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img
from PIL import Image


# Function to convert image to black and white with a resolution of 28x28
def preprocess_image(file_path):
    img = Image.open(file_path).convert('L')  # Convert to grayscale
    img = img.resize((28, 28), Image.ANTIALIAS)  # Resize to 28x28
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

# Define the source and destination folders
source_folder = '/content/drive/MyDrive/drone_data/new_images'
destination_folder = '/content/drive/My Drive/drone_data/new_images/processed_images'

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Iterate through each image in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        file_path = os.path.join(source_folder, filename)
        # Preprocess the image
        processed_img_array = preprocess_image(file_path)

        # Save the processed image to the destination folder using cv2
        processed_img_path = os.path.join(destination_folder, f"bw_{filename}")
        cv2.imwrite(processed_img_path, processed_img_array * 255)

print("Image processing and saving complete.")


In [ ]:
import os
import shutil

# Path to the folder containing your pictures
folder_path = '/content/drive/My Drive/drone_data/new_images/processed_images'

# Create subdirectories if they don't exist
os.makedirs(os.path.join(folder_path, 'drone'), exist_ok=True)
os.makedirs(os.path.join(folder_path, 'no_drone'), exist_ok=True)

# Lists to store pictures
drone = []
no_drone = []

# Iterate through files in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('bw_random'):
        # Move to no_drone folder
        no_drone.append(filename)
        shutil.move(os.path.join(folder_path, filename), os.path.join(folder_path, 'no_drone', filename))
    elif filename.startswith('bw_yolo'):
        # Move to drone folder
        drone.append(filename)
        shutil.move(os.path.join(folder_path, filename), os.path.join(folder_path, 'drone', filename))

# Print the lists
print('drone:', drone)
print('no_drone:', no_drone)


drone: []
no_drone: []


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import os

# Set the path to the main folder containing subfolders
main_folder_path = '/content/drive/My Drive/drone_data/new_images/processed_images'

# List to store image paths and corresponding labels
image_paths = []
labels = []

# Define label classes
class_names = ['no_drone', 'drone']

# Iterate through each class folder
for class_name in class_names:
    class_folder = os.path.join(main_folder_path, class_name)

    # Iterate through each image in the class folder
    for filename in os.listdir(class_folder):
        if filename.endswith('.jpg'):  # assuming the images are in JPG format
            img_path = os.path.join(class_folder, filename)

            # Append the image path and corresponding label to the lists
            image_paths.append(img_path)
            labels.append(class_names.index(class_name))

# Split the data into training and validation sets
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)

# Function to load and preprocess an image
def load_and_preprocess_image(img_path, label):
    # Extract the path from the symbolic tensor
    img_path = img_path.numpy().decode('utf-8')

    img = load_img(img_path, target_size=(256, 256))  # adjust target_size as needed
    img_array = img_to_array(img) / 255.0  # normalize pixel values to [0, 1]
    return img_array, label

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int64)
train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels))
train_dataset = train_dataset.map(lambda x, y: tf.py_function(load_and_preprocess_image, [x, y], [tf.float32, tf.int64]))


val_dataset = tf.data.Dataset.from_tensor_slices((val_image_paths, val_labels))
val_dataset = val_dataset.map(lambda x, y: tf.py_function(load_and_preprocess_image, [x, y], [tf.float32, tf.int64]))
val_dataset = val_dataset.batch(32)

# Display the first batch of images and labels (for verification)
for batch in train_dataset.take(1):
    images, labels = batch
    print(images.shape)  # shape of the images batch
    print(labels)  # corresponding labels




InvalidArgumentError: ignored